# Integrating Instructor with SQLModel

SQLModel is a library designed for interacting with SQL databases from Python code using Python objects. `SQLModel` is based on `Pydantic` and `SQLAlchemy` and was created by tiangolo who also developed `FastAPI`. So you can expect seamless integration across all these libraries, reducing code duplicating and improving your developer experience.

## Example: Adding responses from Instructor directly to your DB

### Defining the Models

First we'll define a model that will serve as a table for our database and the structure of our outputs from `Instructor`

**Model Definition**
You'll need to subclass your models with both `SQLModel` and `instructor.OpenAISchema` for them to work with SQLModel

In [1]:
import instructor
from typing import Optional
from sqlmodel import Field, SQLModel, create_engine

In [2]:
from dotenv import load_dotenv
load_dotenv("../api_keys.env")

True

In [3]:
from openai import OpenAI
client = instructor.from_openai(OpenAI())

### Generating a record

The `create_hero` function will query `OpenAI` for a `Hero` record

In [4]:
class Hero(SQLModel, instructor.OpenAISchema, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

In [5]:
def create_hero() -> Hero:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=Hero,
        messages=[
            {"role": "user", "content": "Make a new superhero"},
        ],
    )

In [6]:
hero = create_hero()
print(hero.model_dump())

{'name': 'Crimson Blaze', 'secret_name': 'Tara Turner', 'age': 28, 'id': None}


### Inserting the response into the DB

In [7]:
engine = create_engine("sqlite:///database.db")
SQLModel.metadata.create_all(engine)

In [10]:
# # BUG: NameError: name 'Session' is not defined

# with Session(engine) as session:
#     session.add(hero)
#     session.commit()

And there you have it! You can now use the same models for your database and `Instructor` enabling them work seamlessly! Also checkout the [FastAPI](https://python.useinstructor.com/concepts/fastapi/) guide to see how you can use these models in an API as well.